## Generating recommendations based on cosine similarity.

In [2]:
import pandas as pd
import numpy as np
import os
import requests
import shutil

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
from keras.models import Model
from keras.applications import vgg16
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics.pairwise import cosine_similarity

%load_ext autotime

/Users/yeongchoi/Anaconda3/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load feature dataset (4k images)

In [4]:
features_df = pd.read_csv('./Results/features_50k.csv')
features_df = features_df.rename(columns={"Unnamed: 0": "product_id"}, errors="raise")
features_df.set_index('product_id',inplace=True)

time: 4.67 s


In [5]:
features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
product_id,,,,,,,,,,,,,,,,,,,,,
B008JFDFIW,0.0,0.0,1.847189,2.641750,0.0,0.000000,0.000000,6.623529,0.0,0.000000,...,0.0,0.0,0.0,1.501545,0.0000,0.000000,0.0,0.0,0.0,1.291022
B00IM5FCOY,0.0,0.0,0.164301,0.000000,0.0,1.673070,0.000000,0.000000,0.0,6.619954,...,0.0,0.0,0.0,0.000000,0.0000,5.276841,0.0,0.0,0.0,0.000000
B00AAHNZUU,0.0,0.0,0.000000,0.855917,0.0,0.000000,0.000000,0.000000,0.0,3.871443,...,0.0,0.0,0.0,0.000000,1.8203,4.572748,0.0,0.0,0.0,0.000000
B004J4WWK4,0.0,0.0,0.000000,0.000000,0.0,3.584921,0.000000,0.000000,0.0,1.201754,...,0.0,0.0,0.0,0.000000,0.0000,6.192037,0.0,0.0,0.0,0.000000
B000BIUAYW,0.0,0.0,0.000000,4.482935,0.0,0.000000,2.517327,0.000000,0.0,3.512050,...,0.0,0.0,0.0,3.508799,0.0000,0.000000,0.0,0.0,0.0,1.227973


time: 30.2 ms


### Calculate cosine similarity 

In [48]:
# all product list in features dataframe
productIds = list(features_df.index)

# compute cosine similarities between images
cosSimilarities = cosine_similarity(features_df)

# store the results into a pandas dataframe
cos_similarities_df = pd.DataFrame(cosSimilarities, columns=productIds, index=productIds)
print('Shape of cosine similarities dataframe:', cos_similarities_df.shape,'\n')
cos_similarities_df.head()

Shape of cosine similarities dataframe: (4249, 4249) 



,B008JFDFIW,B00IM5FCOY,B00AAHNZUU,B004J4WWK4,B000BIUAYW,B00GUV6U1Q,B009HGDS9I,B00026ORRY,B000I66VSG,B00DX9TEAM,...,B00JWY671G,B00NCPIE9Y,B00193SGBC,B000H73YM2,B0002UNNTS,B001CDJXC0,B00NC4F22C,B001OW6T12,B000AJHWCA,B0010T46LE
B008JFDFIW,1.000000,0.362864,0.308780,0.417288,0.432381,0.685618,0.234785,0.457745,0.516445,0.417970,...,0.425732,0.407043,0.460005,0.455167,0.396222,0.502779,0.670686,0.447899,0.268567,0.501026
B00IM5FCOY,0.362864,1.000000,0.360608,0.665957,0.301227,0.437460,0.183060,0.542963,0.350693,0.479026,...,0.340522,0.473009,0.591870,0.527001,0.392424,0.298348,0.469669,0.457021,0.786262,0.404387
B00AAHNZUU,0.308780,0.360608,1.000000,0.375854,0.498847,0.279762,0.208102,0.300166,0.393287,0.375296,...,0.461527,0.270608,0.309120,0.300027,0.436809,0.303264,0.324404,0.293918,0.346671,0.332892
B004J4WWK4,0.417288,0.665957,0.375854,1.000000,0.315326,0.531801,0.217042,0.701494,0.546180,0.544264,...,0.385281,0.528229,0.564211,0.552363,0.486948,0.499572,0.412053,0.553167,0.684340,0.519154
B000BIUAYW,0.432381,0.301227,0.498847,0.315326,1.000000,0.450696,0.291314,0.341520,0.506457,0.488138,...,0.619143,0.374053,0.371834,0.391754,0.540534,0.400048,0.425884,0.353776,0.304871,0.390239


time: 1.23 s


In [49]:
# Save as csv file
cos_similarities_df.to_csv('./Results/cosine_similarities_50K.csv ',index=False)

time: 29.1 s


### Get recommended product result and image urls

In [21]:
# pre-recommeded_result
pre_recommeded_result_df = pd.read_csv('./Datasets/Shoes_for_100_users_per_100_products_prediction_Ver3.csv')
print("shape of pre_recommended result dataframe:", pre_recommeded_result_df.shape)

shape of pre_recommended result dataframe: (10000, 7)
time: 31.7 ms


In [38]:
# imgUrl_dict
# Get image urls and save as dictionary. {asin: image_url}
imgUrl_df = pd.read_csv('./Results/asin_url_for_50k.csv')
imgUrl_dict = pd.Series(imgUrl_df.url.values,index=imgUrl_df.asin).to_dict()

time: 25.6 ms


In [50]:
## search for top 10 items in each pre-recommended items (from Deap learning Model)
num_top = 100

## recommned top 5 products based on similarity for each pre-recommended item
num_recommend = 5

time: 1.7 ms


In [51]:
def Get_Image_Recommendations_DF(pre_recommeded_result_df, cos_similarities_df, imgUrl_dict, num_top, num_recommend):

    # Create empty dataframe
    column_names = ["user_id","given_product", "given_product_img_url", \
                    "recommended_product", "recommended_product_img_url",\
                    "similarity_score"]

    recommeded_df = pd.DataFrame(columns = column_names)

    # 100 users
    users_100 = list(np.unique(pre_recommeded_result_df['user_id'].to_list()))

    # all unique products in model
    all_products = list(cos_similarities_df.columns)

    # get recommeded product for each user 
    for usr in users_100:
        recommended_100 = pre_recommeded_result_df.loc[pre_recommeded_result_df['user_id'] == usr]
        products = recommended_100['asin'].to_list()

        for product in products[:num_top]: ## search for top 10 items in each pre-recommended items
    
            if product not in all_products:
                continue
        
            # Sort product by similarity score. 
            sorted_cos_similarities = cos_similarities_df[product].sort_values(ascending=False)
    
            # Drop duplicated similarity score.
            sorted_cos_similarities = sorted_cos_similarities.drop_duplicates(keep='first', inplace=False)
    
            # select similiarty score less than 0.95. 
            sorted_cos_similarities = sorted_cos_similarities[sorted_cos_similarities < 0.95]
                
            asins = list(sorted_cos_similarities.index)
            scores = list(sorted_cos_similarities.values)
        
            for i in range(num_recommend): # recommned top 5 products based on similarity for each pre-recommended item
                new_row = { "user_id": usr,\
                            "given_product": product,\
                            "given_product_img_url": imgUrl_dict[product],\
                            "recommended_product": asins[i], \
                            "recommended_product_img_url": imgUrl_dict[asins[i]],\
                            "similarity_score": scores[i] }
            
                recommeded_df = recommeded_df.append(new_row, ignore_index=True)
    
            break 
    
    return recommeded_df
        

time: 11.5 ms


In [52]:
recommeded_df = Get_Image_Recommendations_DF(pre_recommeded_result_df, cos_similarities_df, imgUrl_dict, num_top, num_recommend)
recommeded_df.head()

,user_id,given_product,given_product_img_url,recommended_product,recommended_product_img_url,similarity_score
0,19,B000ARPN28,https://images-na.ssl-images-amazon.com/images...,B003GDJ1PS,https://images-na.ssl-images-amazon.com/images...,0.833958
1,19,B000ARPN28,https://images-na.ssl-images-amazon.com/images...,B002B4A954,https://images-na.ssl-images-amazon.com/images...,0.815940
2,19,B000ARPN28,https://images-na.ssl-images-amazon.com/images...,B000BMT90E,https://images-na.ssl-images-amazon.com/images...,0.813858
3,19,B000ARPN28,https://images-na.ssl-images-amazon.com/images...,B007WTJCY2,https://images-na.ssl-images-amazon.com/images...,0.809082
4,19,B000ARPN28,https://images-na.ssl-images-amazon.com/images...,B000ARTQFI,https://images-na.ssl-images-amazon.com/images...,0.798094


time: 1.81 s


In [53]:
print("shape of recommended result:", recommeded_df.shape)

# Unique Users
print("unique users in recommended result:", len(np.unique(recommeded_df['user_id'].to_list())))

# Unique Given Products
given_products = list(np.unique(recommeded_df['given_product'].to_list()))
print("unique given products in recommended result", len(given_products))

## Unique Recommended Products
recommended_products = list(np.unique(recommeded_df['recommended_product'].to_list()))
print("unique recommended products in recommended result:",len(recommended_products))

shape of recommended result: (500, 6)
unique users in recommended result: 100
unique given products in recommended result 98
unique recommended products in recommended result: 431
time: 4.67 ms


In [54]:
# save to csv file
recommeded_df.to_csv('./Predictions/RESULT_image_recommended_product_50k_for_100JH_ver3_all.csv',index=False)

time: 14.8 ms


### Get Image folder for given and recommended products 

In [50]:
# Image folder path for given products 
given_Image_path = "./Recommended_Images_for_100_user/Images_100JH_ver3_top5/given_products_100JH_ver3_top5/"

for asin in given_products:
    
    url = imgUrl_dict[asin]
    # Open the url image, set stream to True, this will return the stream content.
    resp = requests.get(url, stream=True)
    # Open a local file with wb ( write binary ) permission. (created increment filename)
    path = os.path.join(given_Image_path, asin + "." + "jpg")
#     print(path)
    local_file = open(path, 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    del resp

time: 3.31 s


In [51]:
# Image folder path for recommended products 
recommended_Image_path = "./Recommended_Images_for_100_user/Images_100JH_ver3_top5/recommended_products_100JH_ver3_top5/"

for asin in recommended_products:
    
    url = imgUrl_dict[asin]
    # Open the url image, set stream to True, this will return the stream content.
    resp = requests.get(url, stream=True)
    # Open a local file with wb ( write binary ) permission. (created increment filename)
    path = os.path.join(recommended_Image_path, asin + "." + "jpg")
#     print(path)
    local_file = open(path, 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    del resp

time: 18.6 s
